In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import stan # python3 -m pip install pystan
import nest_asyncio
nest_asyncio.apply()
import warnings

In [3]:
df = pd.read_csv("data.csv", header=5, index_col=0)
df = df.shift(axis=1)
df.head()

,Region Code,Region Name,Country Code,Country Name,Year,Sex,Age group code,Age Group,Number,Percentage of cause-specific deaths out of total deaths,Age-standardized death rate per 100 000 standard population,Death rate per 100 000 population
EU,None,Europe,ALB,Albania,1987,All,Age_unknown,[Unknown],0.0,NaN,NaN,NaN
EU,None,Europe,ALB,Albania,1987,All,Age85_over,[85+],2.0,0.089847,NaN,20.000000
EU,None,Europe,ALB,Albania,1987,All,Age80_84,[80-84],0.0,0.000000,NaN,0.000000
EU,None,Europe,ALB,Albania,1987,All,Age75_79,[75-79],6.0,0.283688,NaN,19.480519
EU,None,Europe,ALB,Albania,1987,All,Age70_74,[70-74],9.0,0.498339,NaN,22.058824


In [4]:
df = df[df['Country Code'] == 'USA']
df = df[df['Year'] > 2011]
print(df)

    Region Code                      Region Name Country Code  \
NAC        None  North America and the Caribbean          USA   
NAC        None  North America and the Caribbean          USA   
NAC        None  North America and the Caribbean          USA   
NAC        None  North America and the Caribbean          USA   
NAC        None  North America and the Caribbean          USA   
..          ...                              ...          ...   
NAC        None  North America and the Caribbean          USA   
NAC        None  North America and the Caribbean          USA   
NAC        None  North America and the Caribbean          USA   
NAC        None  North America and the Caribbean          USA   
NAC        None  North America and the Caribbean          USA   

                 Country Name  Year     Sex Age group code  Age Group  \
NAC  United States of America  2012     All    Age_unknown  [Unknown]   
NAC  United States of America  2012     All        Age_all      [All]   


In [5]:

stan_code = '''
data {
  int<lower=0> N; // number of data points
  vector[N] x;    // observation year
  vector[N] y;    // observation number of drowned
  real xpred;     // prediction year
  real tau;
}
parameters {
  real alpha;
  real beta;
  real<lower=0> sigma;
}
transformed parameters {
  vector[N] mu;
  mu = alpha + beta * x;
}
model {
  beta ~ normal(0, tau*tau);
  y ~ normal(mu, sigma);
}
generated quantities {
  real ypred;
  ypred = normal_rng(alpha + beta * xpred, sigma);
}
'''
years = [1, 2, 3, 4, 5]
drowning = [12, 25, 73, 43, 45]

data = dict(
    N=len(years),
    x=years,
    y=drowning,
    xpred=2019,
    tau=26.612146647843897,
)

stan_model = stan.build(stan_code, data=data)
warnings.filterwarnings('ignore') 


Building...

Error handling request
Traceback (most recent call last):
  File "/opt/software/lib/python3.10/site-packages/aiohttp/web_protocol.py", line 452, in _handle_request
    resp = await request_handler(request)
  File "/opt/software/lib/python3.10/site-packages/aiohttp/web_app.py", line 543, in _handle
    resp = await handler(request)
  File "/opt/software/lib/python3.10/site-packages/httpstan/views.py", line 103, in handle_create_model
    _, stanc_warnings = httpstan.compile.compile(program_code, stan_model_name)
  File "/opt/software/lib/python3.10/site-packages/httpstan/compile.py", line 37, in compile
    completed_process = subprocess.run(run_args, capture_output=True, timeout=1)
  File "/opt/software/lib/python3.10/subprocess.py", line 503, in run
    with Popen(*popenargs, **kwargs) as process:
  File "/opt/software/lib/python3.10/subprocess.py", line 971, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "/opt/software/lib/python3.10/subprocess.py

ValueError: The JSON document has an improper structure: missing or superfluous commas, braces, missing keys, etc.

In [19]:
fit = stan_model.sample(num_chains=4, num_samples=1000)

Sampling:   0%
Sampling:  25% (2000/8000)
Sampling:  50% (4000/8000)
Sampling:  75% (6000/8000)
Sampling: 100% (8000/8000)
Sampling: 100% (8000/8000), done.
Messages received during sampling:
  Gradient evaluation took 5.5e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.55 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 3.4e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.34 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 5.1e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.51 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 6.3e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.63 seconds.
  Adjust your expectations accordingly!


In [20]:
df = fit.to_frame()
df

parameters,lp__,accept_stat__,stepsize__,treedepth__,n_leapfrog__,divergent__,energy__,alpha,beta,sigma,mu.1,mu.2,mu.3,mu.4,mu.5,ypred
draws,,,,,,,,,,,,,,,,
0,-16.153349,0.975258,0.127810,2.0,7.0,0.0,17.277751,-33.202630,26.060460,31.994878,-7.142170,18.918290,44.978751,71.039211,97.099672,52624.138417
1,-15.607782,0.567257,0.072863,5.0,31.0,0.0,18.157437,-8.573160,19.844161,35.173949,11.271002,31.115163,50.959324,70.803486,90.647647,40027.675709
2,-14.019648,0.992140,0.091252,4.0,15.0,0.0,14.042663,21.901654,7.610174,22.995427,29.511828,37.122002,44.732176,52.342350,59.952524,15358.148470
3,-20.021200,0.994610,0.110082,4.0,23.0,0.0,22.961131,11.234473,7.599791,147.851931,18.834264,26.434055,34.033846,41.633638,49.233429,15441.834232
4,-17.423417,0.980505,0.127810,3.0,15.0,0.0,20.532015,-61.721421,35.539269,47.514015,-26.182151,9.357118,44.896388,80.435657,115.974926,71611.011805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,-16.039204,0.604752,0.110082,3.0,15.0,0.0,17.693800,-7.735407,12.245397,49.428129,4.509990,16.755387,29.000783,41.246180,53.491577,24730.644789
3996,-15.477930,0.989430,0.127810,2.0,3.0,0.0,20.153714,-27.060187,19.186581,35.673168,-7.873606,11.312975,30.499556,49.686137,68.872718,38765.039747
3997,-21.206885,0.974803,0.072863,3.0,15.0,0.0,26.235092,-0.311788,23.352531,195.332655,23.040743,46.393275,69.745806,93.098337,116.450869,47034.879692
